<a href="https://colab.research.google.com/github/Frost-04/ML-model-on-F1-Dataset-to-predict-podium-using-SVM/blob/main/Formula_1_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [290]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [291]:
!wget -O results.csv "https://drive.google.com/uc?id=1jvzxG-lDm9AZ6g8vrmy8YknCbmMsmifN"
!wget -O driver_standings.csv "https://drive.google.com/uc?id=1cDTw95ehqpC8UksDLspQreogYKQCOi0n"
!wget -O drivers.csv "https://drive.google.com/uc?id=1Qm7tbxM_A7szNo6H-rO487oiJYYuDBqP"
file_path = '/content/results.csv'
results = pd.read_csv(file_path)
file_path = '/content/driver_standings.csv'
driver_standings=pd.read_csv(file_path)
file_path = '/content/drivers.csv'
drivers=pd.read_csv(file_path)



--2024-12-12 15:06:51--  https://drive.google.com/uc?id=1jvzxG-lDm9AZ6g8vrmy8YknCbmMsmifN
Resolving drive.google.com (drive.google.com)... 74.125.23.139, 74.125.23.100, 74.125.23.101, ...
Connecting to drive.google.com (drive.google.com)|74.125.23.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1jvzxG-lDm9AZ6g8vrmy8YknCbmMsmifN [following]
--2024-12-12 15:06:51--  https://drive.usercontent.google.com/download?id=1jvzxG-lDm9AZ6g8vrmy8YknCbmMsmifN
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 64.233.188.132, 2404:6800:4008:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|64.233.188.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1717126 (1.6M) [application/octet-stream]
Saving to: ‘results.csv’

results.csv         100%[===================>]   1.64M  --.-KB/s    in 0.009s  

2024-12-12 15:06:53 (173 MB/s) - ‘resu

# Drivers table pre-processing

In [292]:
print("NULL check:------\n")
print(drivers.isnull().sum())
print("Data types:------\n")
print(drivers.dtypes)
drivers

NULL check:------

driverId       0
driverRef      0
number         0
code           0
forename       0
surname        0
dob            0
nationality    0
url            0
dtype: int64
Data types:------

driverId        int64
driverRef      object
number         object
code           object
forename       object
surname        object
dob            object
nationality    object
url            object
dtype: object


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...,...
855,857,piastri,81,PIA,Oscar,Piastri,2001-04-06,Australian,http://en.wikipedia.org/wiki/Oscar_Piastri
856,858,sargeant,2,SAR,Logan,Sargeant,2000-12-31,American,http://en.wikipedia.org/wiki/Logan_Sargeant
857,859,lawson,40,LAW,Liam,Lawson,2002-02-11,New Zealander,http://en.wikipedia.org/wiki/Liam_Lawson
858,860,bearman,38,BEA,Oliver,Bearman,2005-05-08,British,http://en.wikipedia.org/wiki/Oliver_Bearman


In [293]:
#One hot encoding for DOB
drivers['dob'] = pd.to_datetime(drivers['dob'])
drivers['birthYear'] = drivers['dob'].dt.year
drivers['birthMonth'] = drivers['dob'].dt.month
drivers['birthDay'] = drivers['dob'].dt.day

drivers = drivers.drop(columns=['url', 'driverRef','number','dob']) #driverRef removed as its 1:1 corr with code, number is not required as its unique for each column but with null values logically it has direct corelation with code


In [294]:
drivers

,driverId,code,forename,surname,nationality,birthYear,birthMonth,birthDay
0,1,HAM,Lewis,Hamilton,British,1985,1,7
1,2,HEI,Nick,Heidfeld,German,1977,5,10
2,3,ROS,Nico,Rosberg,German,1985,6,27
3,4,ALO,Fernando,Alonso,Spanish,1981,7,29
4,5,KOV,Heikki,Kovalainen,Finnish,1981,10,19
...,...,...,...,...,...,...,...,...
855,857,PIA,Oscar,Piastri,Australian,2001,4,6
856,858,SAR,Logan,Sargeant,American,2000,12,31
857,859,LAW,Liam,Lawson,New Zealander,2002,2,11
858,860,BEA,Oliver,Bearman,British,2005,5,8


# Results table pre-processing

In [295]:
print("NULL check:------\n")
print(results.isnull().sum())
print("Data types check:------\n")
print(results.dtypes)
results

NULL check:------

resultId           0
raceId             0
driverId           0
constructorId      0
number             0
grid               0
position           0
positionText       0
positionOrder      0
points             0
laps               0
time               0
milliseconds       0
fastestLap         0
rank               0
fastestLapTime     0
fastestLapSpeed    0
statusId           0
dtype: int64
Data types check:------

resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
dtype: object


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26694,26700,1141,832,6,55,0,\N,R,16,0.0,38,\N,\N,19,17,1:24.201,184.230,3
26695,26701,1141,861,3,43,16,\N,R,17,0.0,30,\N,\N,21,18,1:24.296,184.022,3
26696,26702,1141,848,3,23,7,\N,W,18,0.0,0,\N,\N,\N,0,\N,\N,54
26697,26703,1141,840,117,18,10,\N,W,19,0.0,0,\N,\N,\N,0,\N,\N,54


In [ ]:
# Function to convert fastestLapTime string to seconds
def convert_to_seconds(value):
    if value == r'\N':  # Handle null values
        return np.nan
    minutes, seconds = map(float, value.split(':'))
    return minutes * 60 + seconds

# Apply the function to the column
results['fastestLapTime'] = results['fastestLapTime'].apply(convert_to_seconds)
# Ensure the column is of type float
results['fastestLapTime'] = results['fastestLapTime'].astype(float)

# milliseconds to seconds
results['milliseconds'] = results['milliseconds'].replace(r'\\N', np.nan, regex=True)
# Convert the column to numeric (this will convert invalid values to NaN)
results['milliseconds'] = pd.to_numeric(results['milliseconds'], errors='coerce')
# Convert milliseconds to seconds (float)
results['milliseconds'] = results['milliseconds'] / 1000
# Ensure the column is of type float
results['milliseconds'] = results['milliseconds'].astype(float)

#Convert number to integer
results['number'] = results['number'].replace(r'\\N', np.nan, regex=True)
results['number'] = results['number'].astype(float)
results = results.dropna(subset=['number'])


#convert fastestLapSpeed to float
results['fastestLapSpeed'] = results['fastestLapSpeed'].replace(r'\\N', np.nan, regex=True)
results['fastestLapSpeed'] = results['fastestLapSpeed'].astype(float)

#convert Rank to number
results['rank'] = results['rank'].replace(r'\\N', np.nan, regex=True)
results['rank'] = pd.to_numeric(results['rank'], errors='coerce')

#convert fastestLap to number
results['fastestLap'] = results['fastestLap'].replace(r'\\N', np.nan, regex=True)
results['fastestLap'] = pd.to_numeric(results['fastestLap'], errors='coerce')


#Dropping irrelevant and derived columns and renaming properly
results = results.drop(columns=['position','positionText','time'])
results = results.rename(columns={'positionOrder': 'Position', 'milliseconds': 'completionTime'})
#Dropping because 80% values are NaN 18k+ rows
results = results.drop(columns=['rank'])


In [297]:
#Finding a candidate key for merging

#Checking if raceId and driverId together make a key
is_unique_combination = not results.duplicated(subset=['raceId', 'driverId']).any()
print("Is the combination of columns 'A' and 'B' unique?", is_unique_combination , '\n')

#finding duplicated rows in reference to raceId and driverId
print("Duplicated Rows are")
duplicates = results[results.duplicated(subset=['raceId', 'driverId'], keep=False)]
print(duplicates)

#Since there are 176 duplicated rows we can safely remove them
results = results[~results.duplicated(subset=['raceId', 'driverId'], keep=False)]

#Checking if raceId and driverId together make a key
is_unique_combination = not results.duplicated(subset=['raceId', 'driverId']).any()
print("\nIs the combination of columns 'A' and 'B' unique?", is_unique_combination)


Is the combination of columns 'A' and 'B' unique? False 

Duplicated Rows are
       resultId  raceId  driverId  constructorId  number  grid  Position  \
13188     13189     540       229             54    10.0     0        26   
13191     13192     540       229             57    23.0     0        29   
17362     17363     717       373            172     1.0     1         7   
17740     17741     733       465            172    48.0     0        22   
17743     17744     733       465             97    50.0     0        25   
...         ...     ...       ...            ...     ...   ...       ...   
20293     20296     770       479            118    12.0    10        14   
20294     20297     774       566            105    32.0    11         4   
20295     22372     746       475            170    38.0     8         3   
24297     24303     745       418            172    21.0    15        11   
24298     24304     717       373            172     2.0     1        12   

       po

In [298]:
results.describe()

,resultId,raceId,driverId,constructorId,number,grid,Position,points,laps,completionTime,fastestLap,fastestLapTime,fastestLapSpeed,statusId
count,26517.000000,26517.000000,26517.000000,26517.000000,26517.000000,26517.000000,26517.000000,26517.000000,26517.000000,7618.000000,8195.000000,8195.000000,8195.000000,26517.000000
mean,13307.879474,548.705170,275.680695,49.833201,18.073425,11.146849,12.795716,1.990983,46.129464,6173.209927,42.722514,90.833155,203.965808,17.185051
std,7717.027207,312.774559,281.769476,61.426839,15.503553,7.197765,7.665577,4.352474,29.122680,1638.609244,16.626873,12.374142,21.364154,25.973839
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.526000,1.000000,55.404000,89.540000,1.000000
25%,6630.000000,298.000000,57.000000,6.000000,7.000000,5.000000,6.000000,0.000000,23.000000,5381.298000,33.000000,80.782500,193.250500,1.000000
50%,13261.000000,527.000000,170.000000,25.000000,16.000000,11.000000,12.000000,0.000000,53.000000,5789.590000,46.000000,90.294000,204.783000,10.000000
75%,19976.000000,809.000000,385.000000,59.000000,24.000000,17.000000,18.000000,2.000000,66.000000,6399.354500,54.000000,99.543000,217.083000,14.000000
max,26704.000000,1141.000000,861.000000,215.000000,208.000000,34.000000,39.000000,50.000000,200.000000,15090.540000,85.000000,202.300000,257.320000,141.000000




---



Result pre-processing completed

## Pre-Processing Driver_Standings

In [299]:
print("NULL check:------\n")
print(driver_standings.isnull().sum())
print("\nData types:------")
print(driver_standings.dtypes)
driver_standings

NULL check:------

driverStandingsId    0
raceId               0
driverId             0
points               0
position             0
positionText         0
wins                 0
dtype: int64

Data types:------
driverStandingsId      int64
raceId                 int64
driverId               int64
points               float64
position               int64
positionText          object
wins                   int64
dtype: object


,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0
...,...,...,...,...,...,...,...
34788,73157,1141,832,244.0,5,5,2
34789,73156,1141,815,151.0,8,8,0
34790,73155,1141,830,393.0,1,1,8
34791,73176,1141,861,5.0,19,19,0


In [300]:
driver_standings = driver_standings.rename(columns={'points': 'totalPoints'})
driver_standings = driver_standings.rename(columns={'position': 'overallPosition'})

#Removing as its same as position
driver_standings=driver_standings.drop(columns=['positionText'])

In [301]:
driver_standings.describe()

#Finding key for merging tables
is_unique_combination = not driver_standings.duplicated(subset=['raceId', 'driverId']).any()

print("Is the combination of columns 'A' and 'B' unique?", is_unique_combination)

Is the combination of columns 'A' and 'B' unique? True



---
**BASIC PRE-PROCESSING COMPLETED**




## Merging Datasets

In [302]:
print(results.describe())
print("\n------------------------------\n")
print(drivers.describe())
print("\n------------------------------\n")
print(driver_standings.describe())

           resultId        raceId      driverId  constructorId        number  \
count  26517.000000  26517.000000  26517.000000   26517.000000  26517.000000   
mean   13307.879474    548.705170    275.680695      49.833201     18.073425   
std     7717.027207    312.774559    281.769476      61.426839     15.503553   
min        1.000000      1.000000      1.000000       1.000000      0.000000   
25%     6630.000000    298.000000     57.000000       6.000000      7.000000   
50%    13261.000000    527.000000    170.000000      25.000000     16.000000   
75%    19976.000000    809.000000    385.000000      59.000000     24.000000   
max    26704.000000   1141.000000    861.000000     215.000000    208.000000   

               grid      Position        points          laps  completionTime  \
count  26517.000000  26517.000000  26517.000000  26517.000000     7618.000000   
mean      11.146849     12.795716      1.990983     46.129464     6173.209927   
std        7.197765      7.665577   

In [303]:
common_columns = set(results.columns).intersection(set(drivers.columns))

print("Common columns between results and drivers:", common_columns)

Common columns between results and drivers: {'driverId'}


In [304]:
result_driver_merge = pd.merge(results, drivers, on='driverId', how='inner')
result_driver_merge

,resultId,raceId,driverId,constructorId,number,grid,Position,points,laps,completionTime,...,fastestLapTime,fastestLapSpeed,statusId,code,forename,surname,nationality,birthYear,birthMonth,birthDay
0,1,18,1,1,22.0,1,1,10.0,58,5690.616,...,87.452,218.300,1,HAM,Lewis,Hamilton,British,1985,1,7
1,2,18,2,2,3.0,5,2,8.0,58,5696.094,...,87.739,217.586,1,HEI,Nick,Heidfeld,German,1977,5,10
2,3,18,3,3,7.0,7,3,6.0,58,5698.779,...,88.090,216.719,1,ROS,Nico,Rosberg,German,1985,6,27
3,4,18,4,4,5.0,11,4,5.0,58,5707.797,...,88.603,215.464,1,ALO,Fernando,Alonso,Spanish,1981,7,29
4,5,18,5,1,23.0,3,5,4.0,58,5708.630,...,87.418,218.385,1,KOV,Heikki,Kovalainen,Finnish,1981,10,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26512,26700,1141,832,6,55.0,0,16,0.0,38,NaN,...,84.201,184.230,3,SAI,Carlos,Sainz,Spanish,1994,9,1
26513,26701,1141,861,3,43.0,16,17,0.0,30,NaN,...,84.296,184.022,3,COL,Franco,Colapinto,Argentinian,2003,5,27
26514,26702,1141,848,3,23.0,7,18,0.0,0,NaN,...,NaN,NaN,54,ALB,Alexander,Albon,Thai,1996,3,23
26515,26703,1141,840,117,18.0,10,19,0.0,0,NaN,...,NaN,NaN,54,STR,Lance,Stroll,Canadian,1998,10,29


In [305]:
common_columns = set(result_driver_merge.columns).intersection(set(driver_standings.columns))

print("Common columns:", common_columns)

Common columns: {'driverId', 'raceId'}


In [306]:
datasetmain =pd.merge(result_driver_merge, driver_standings, on=['raceId','driverId'], how='inner')
datasetmain

,resultId,raceId,driverId,constructorId,number,grid,Position,points,laps,completionTime,...,forename,surname,nationality,birthYear,birthMonth,birthDay,driverStandingsId,totalPoints,overallPosition,wins
0,1,18,1,1,22.0,1,1,10.0,58,5690.616,...,Lewis,Hamilton,British,1985,1,7,1,10.0,1,1
1,2,18,2,2,3.0,5,2,8.0,58,5696.094,...,Nick,Heidfeld,German,1977,5,10,2,8.0,2,0
2,3,18,3,3,7.0,7,3,6.0,58,5698.779,...,Nico,Rosberg,German,1985,6,27,3,6.0,3,0
3,4,18,4,4,5.0,11,4,5.0,58,5707.797,...,Fernando,Alonso,Spanish,1981,7,29,4,5.0,4,0
4,5,18,5,1,23.0,3,5,4.0,58,5708.630,...,Heikki,Kovalainen,Finnish,1981,10,19,5,4.0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26043,26700,1141,832,6,55.0,0,16,0.0,38,NaN,...,Carlos,Sainz,Spanish,1994,9,1,73157,244.0,5,2
26044,26701,1141,861,3,43.0,16,17,0.0,30,NaN,...,Franco,Colapinto,Argentinian,2003,5,27,73176,5.0,19,0
26045,26702,1141,848,3,23.0,7,18,0.0,0,NaN,...,Alexander,Albon,Thai,1996,3,23,73169,12.0,16,0
26046,26703,1141,840,117,18.0,10,19,0.0,0,NaN,...,Lance,Stroll,Canadian,1998,10,29,73164,24.0,13,0


**Merging Completed**

# EDA starts Here